In [1]:
import numpy as np ; import pandas as pd

In [2]:
data = pd.DataFrame({
    'x0' : [1, 2, 3, 4, 5],
    'x1' : [0.01, -0.01, 0.25, -4.1, 0.],
    'y' : [-1.5, 0., 3.6, 1.3, -2.]
})
data

x0    x1    y
0   1  0.01 -1.5
1   2 -0.01  0.0
2   3  0.25  3.6
3   4 -4.10  1.3
4   5  0.00 -2.0

In [3]:
import patsy

In [6]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [7]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [8]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [9]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [10]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [13]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [14]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

<ipython-input-14-ef4372856a50>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


In [15]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [16]:
y. X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [17]:
new_data = pd.DataFrame({
    'x0' : [6, 7, 8, 9],
    'x1' : [3.1, -0.5, 0, 2.3],
    'y' : [1, 2, 3, 4]
})


In [18]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  x0    x1
           1   6   3.1
           1   7  -0.5
           1   8   0.0
           1   9   2.3
   Terms:
     'Intercept' (column 0)
     'x0' (column 1)
     'x1' (column 2)]

In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [20]:
def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [21]:
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size = N),
         dnorm(0, 0.6, size = N),
         dnorm(0, 0.2, size = N)]
eps = dnorm(0, 0.1, size = N)
beta = [0.1, 0.3, 0.5]
y = np.dot(X, beta) + eps

In [22]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [23]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [24]:
model  =sm.OLS(y, X)

In [25]:
results = model.fit()

In [26]:
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [27]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Thu, 03 Mar 2022   Prob (F-statistic):                    7.44e-12
Time:                        16:54:57   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------